In [1]:
import logging
logging.getLogger('tensorflow').disabled = True
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
!pip install bert-tensorflow

    100% |████████████████████████████████| 71kB 1.2MB/s ta 0:00:011


In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [4]:
import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import os
import nltk
import pickle

sys.path.append('../src')

from commons import elastic
from langdetect import detect
from text_processing import text_normalizer
from importlib import reload
from text_processing import duplicate_finder
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
import Levenshtein
text_normalizer = reload(text_normalizer)
from dateutil.parser import parse
from utilities import excel_writer, excel_reader
from sklearn.model_selection import train_test_split

def get_train_and_test_data_splitted(temp_df, all_dataset = False, test_size=0.05):
    if all_dataset:
        train_data = temp_df
        test_data = temp_df
    else:
        train_data_x, test_data_x,train_y,test_y = train_test_split([data[:-1] for data in temp_df.values], [data[-1] for data in temp_df.values], test_size=test_size)
        train_data = np.concatenate((train_data_x,[[val]for val in train_y]),axis=1)
        test_data = np.concatenate((test_data_x,[[val] for val in test_y]),axis=1)
    return pd.DataFrame(train_data, columns=temp_df.columns), pd.DataFrame(test_data, columns=temp_df.columns)

/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float

In [5]:
!pip install scikit-multilearn==0.2.0

    100% |████████████████████████████████| 92kB 1.3MB/s ta 0:00:011


In [5]:
from sklearn.metrics import f1_score
from text_processing import text_normalizer

def get_f1_score_test_data(test_data, intervention_labeler):
    res_pred, res_prob = intervention_labeler.predict_class(test_data.values, return_probs=True)
    res_true = test_data["Label"].values
    return f1_score(res_true, res_pred, average="macro")

def get_f1_multi_label(test_labels, y_true):
    test_labels = np.concatenate([np.asarray(labels)[:, np.newaxis] for labels in test_labels], axis=1)
    final_res = []
    for i in range(test_labels.shape[0]):
        identified_labels = set()
        for j in range(test_labels.shape[1]):
            if test_labels[i][j] >= 0.5:
                identified_labels.add(j+1)
        if not identified_labels:
            identified_labels.add(12)
        final_res.append(identified_labels)
    cnt = 0
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_all_found = 0
    for idx, res in enumerate(y_true):
        cnt += 1
        cnt_intersect = len(y_true[idx].intersection(final_res[idx]))
        cnt_correct = len(y_true[idx])
        cnt_found = len(final_res[idx])
        precision = 0 if cnt_found == 0 else (cnt_intersect/cnt_found)
        recall = 0 if cnt_correct == 0 else (cnt_intersect/cnt_correct)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        if recall > 0.99:
            cnt_all_found += 1
        cnt_f1 += f1
    return cnt_f1/cnt

def get_f1_multi_label_new(test_labels, y_true):
    final_res = []
    for i in range(len(test_labels)):
        identified_labels = set()
        for j in range(len(test_labels[i])):
            if test_labels[i][j] >= 0.5:
                identified_labels.add(j+1)
        if not identified_labels:
            identified_labels.add(12)
        final_res.append(identified_labels)
    cnt = 0
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_all_found = 0
    for idx, res in enumerate(y_true):
        cnt += 1
        cnt_intersect = len(y_true[idx].intersection(final_res[idx]))
        cnt_correct = len(y_true[idx])
        cnt_found = len(final_res[idx])
        precision = 0 if cnt_found == 0 else (cnt_intersect/cnt_found)
        recall = 0 if cnt_correct == 0 else (cnt_intersect/cnt_correct)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        if recall > 0.99:
            cnt_all_found += 1
        cnt_f1 += f1
    return cnt_f1/cnt

def get_accuracy_multi_label(test_labels, y_true):
    test_labels = np.concatenate([np.asarray(labels)[:, np.newaxis] for labels in test_labels], axis=1)
    final_res = []
    for i in range(test_labels.shape[0]):
        identified_labels = set()
        for j in range(test_labels.shape[1]):
            if test_labels[i][j] >= 0.5:
                identified_labels.add(j+1)
        if not identified_labels:
            identified_labels.add(12)
        final_res.append(identified_labels)
    cnt = 0
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_all_found = 0
    for idx, res in enumerate(y_true):
        cnt += 1
        cnt_intersect = len(y_true[idx].intersection(final_res[idx]))
        cnt_correct = len(y_true[idx])
        cnt_found = len(final_res[idx])
        precision = 0 if cnt_found == 0 else (1 if cnt_intersect > 0 else 0)
        recall = 0 if cnt_correct == 0 else (1 if cnt_intersect > 0 else 0)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        if recall > 0.99:
            cnt_all_found += 1
        cnt_f1 += f1
    return cnt_f1/cnt

def get_accuracy_multi_label_new(test_labels, y_true):
    final_res = []
    for i in range(len(test_labels)):
        identified_labels = set()
        for j in range(len(test_labels[i])):
            if test_labels[i][j] >= 0.5:
                identified_labels.add(j+1)
        if not identified_labels:
            identified_labels.add(12)
        final_res.append(identified_labels)
    cnt = 0
    cnt_precision = 0
    cnt_recall = 0
    cnt_f1 = 0
    cnt_all_found = 0
    for idx, res in enumerate(y_true):
        cnt += 1
        cnt_intersect = len(y_true[idx].intersection(final_res[idx]))
        cnt_correct = len(y_true[idx])
        cnt_found = len(final_res[idx])
        precision = 0 if cnt_found == 0 else (1 if cnt_intersect > 0 else 0)
        recall = 0 if cnt_correct == 0 else (1 if cnt_intersect > 0 else 0)
        f1 = 0
        if (precision + recall) > 0:
            f1 = 2*precision*recall/(precision + recall)
        cnt_precision += precision
        cnt_recall += recall
        if recall > 0.99:
            cnt_all_found += 1
        cnt_f1 += f1
    return cnt_f1/cnt

def normalize_full(text):
    return " ".join(sorted(text_normalizer.normalize_sentence(text).split(" ")))

def deduplicate(dfs):
    dicts = {}
    for df in dfs:
        for i in range(len(df)):
            processed = normalize_full(df["Narrow concept"].values[i])
            if processed not in dicts:
                dicts[processed] = set()
            dicts[processed].add(df["Label"].values[i])
    return dicts

In [6]:
train_df = excel_reader.ExcelReader().read_folder("../tmp/usaid_water")
len(train_df)

Read file ../tmp/usaid_water/train_data.xlsx: 0.06s
Processed file ../tmp/usaid_water/train_data.xlsx: 0.03s
Read file ../tmp/usaid_water/labeled_sample_USAID.xlsx: 0.01s
Processed file ../tmp/usaid_water/labeled_sample_USAID.xlsx: 0.00s
Read file ../tmp/usaid_water/PA00JQVN_labeled_non_intervention.xlsx: 0.00s
Processed file ../tmp/usaid_water/PA00JQVN_labeled_non_intervention.xlsx: 0.00s
Read file ../tmp/usaid_water/part_sentences_model_check_Jaron.xlsx: 0.01s
Processed file ../tmp/usaid_water/part_sentences_model_check_Jaron.xlsx: 0.00s
Read file ../tmp/usaid_water/interv_from_sentences.xlsx: 0.19s
Processed file ../tmp/usaid_water/interv_from_sentences.xlsx: 0.10s
Read file ../tmp/usaid_water/data_sample_to_label_JP.xlsx: 0.01s
Processed file ../tmp/usaid_water/data_sample_to_label_JP.xlsx: 0.00s
Read file ../tmp/usaid_water/additionally_labelled_low_prob.xlsx: 0.01s
Processed file ../tmp/usaid_water/additionally_labelled_low_prob.xlsx: 0.00s


8597

In [7]:
from interventions_labeling_lib import usaid_intervention_labels
train_df["Label"] = [usaid_intervention_labels.USAIDInterventionLabels.INTERVENTION_LABEL_TO_NUMBER[train_df["Label"].values[i]] for i in range(len(train_df))]
deduplicated2labels = deduplicate([train_df])
train_df["Label"].value_counts()

1     1201
11    1186
2     1097
10    1006
8      987
7      831
5      575
12     515
9      515
6      486
4      129
3       69
Name: Label, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
x_train_ind, x_test_ind, y_train, y_test = train_test_split(
            train_df.drop("Label", axis=1), train_df["Label"].values,
            test_size=0.1, random_state=1237, stratify=train_df["Label"].values)
train_data = pd.concat([x_train_ind.reset_index(drop=True), pd.DataFrame({"Label": y_train})], axis=1)
test_data = pd.concat([x_test_ind.reset_index(drop=True), pd.DataFrame({"Label": y_test})], axis=1)

In [14]:
from bert_models import base_bert_model_interventions

common_folder = "/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi"
test_labels = []
for label in usaid_intervention_labels.USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL:
    if label != 10:
        print("#### ", usaid_intervention_labels.USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[label])
        train_copy = train_data.copy()
        test_copy = test_data.copy()
        train_copy["Label"] = [1 if label in deduplicated2labels[normalize_full(train_copy["Narrow concept"].values[i])] else 0 for i in range(len(train_copy))]
        test_copy["Label"] = [1 if label in deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[i])] else 0 for i in range(len(test_copy))]
        
        _intervention_labeler = base_bert_model_interventions.BaseBertModelInterventions(
            os.path.join(common_folder, usaid_intervention_labels.USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[label]),
            [0,1], label_column="Label"
        )
        _intervention_labeler.train_model(train_copy, test_copy)
        
        res_prob, res_label, _ = _intervention_labeler.predict_for_df(test_copy)
        test_labels.append([r[1] for r in res_prob])

print("F1 measure: ", get_f1_multi_label(test_labels, [deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[i])] for i in range(len(test_data))]))
print("Accuracy: ",get_accuracy_multi_label(test_labels, [deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[i])] for i in range(len(test_data))]))

####  Water infrastructure
Started tokenizer loading
Used gpu 0
Model partly loaded
Tokenizrer loaded
0
Config is done
1449
Beginning Training!
Training took time  0:00:00.006161
Used for model gpu 1
[[692  15]
 [ 17 136]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       707
           1       0.90      0.89      0.89       153

   micro avg       0.96      0.96      0.96       860
   macro avg       0.94      0.93      0.94       860
weighted avg       0.96      0.96      0.96       860

F1 score:  0.9360689860243829
Used for model gpu 1
####  Sanitation/Hygiene
Started tokenizer loading
Used gpu 0
Model partly loaded
Tokenizrer loaded
0
Config is done
1449
Beginning Training!
Used for model gpu 1
Used for model gpu 1
Training took time  0:09:30.703767
Used for model gpu 1
[[696   6]
 [ 17 141]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       702
           1       0.96    

In [20]:
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
from bert_models import base_bert_model_interventions
base_bert_model_interventions = reload(base_bert_model_interventions)

common_folder = "/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_1"
_intervention_labeler = base_bert_model_interventions.BaseBertModelInterventions(
    common_folder,
    [0 for i in range(9)], label_column="label",
    multilabel=True
)

def prepare_labelled_data(train_data):
    train_copy = train_data.copy()
    train_copy["label"] = ""
    for i in range(len(train_copy)):
        real_labels = deduplicated2labels[normalize_full(train_copy["Narrow concept"].values[i])]
        real_labels_idx = []
        for idx in range(1, 10):
            real_labels_idx.append(int(idx in real_labels))
        train_copy["label"].values[i] = real_labels_idx
    return train_copy

train_copy = prepare_labelled_data(train_data)
test_copy = prepare_labelled_data(test_data)

_intervention_labeler.train_model(train_copy, test_copy)

res_prob, res_label, _ = _intervention_labeler.predict_for_df(test_copy)

print("F1 measure: ", get_f1_multi_label_new(res_prob, [deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[i])] for i in range(len(test_data))]))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, [deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[i])] for i in range(len(test_data))]))

Started tokenizer loading
Used gpu 0
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0213 20:19:37.179166 139899362359104 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3bdda9f518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0213 20:19:37.528754 139899362359104 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3bdda9f518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 7733


...

INFO:tensorflow:Running local_init_op.


I0213 20:19:42.330813 139899362359104 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0213 20:19:42.401420 139899362359104 session_manager.py:493] Done running local_init_op.


F1 measure:  0.8425193798449607
Accuracy:  0.8732558139534884


In [9]:
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
from bert_models import base_bert_model_interventions
base_bert_model_interventions = reload(base_bert_model_interventions)

common_folder = "/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_2"
_intervention_labeler = base_bert_model_interventions.BaseBertModelInterventions(
    common_folder,
    [0 for i in range(11)], label_column="label",
    multilabel=True
)

def prepare_labelled_data(train_data):
    train_copy = train_data.copy()
    train_copy["label"] = ""
    for i in range(len(train_copy)):
        real_labels = deduplicated2labels[normalize_full(train_copy["Narrow concept"].values[i])]
        real_labels_idx = []
        for idx in range(1, 12):
            real_labels_idx.append(int(idx in real_labels))
        train_copy["label"].values[i] = real_labels_idx
    return train_copy

train_copy = prepare_labelled_data(train_data)
test_copy = prepare_labelled_data(test_data)

_intervention_labeler.train_model(train_copy, test_copy)

res_prob, res_label, _ = _intervention_labeler.predict_for_df(test_copy)

print("F1 measure: ", get_f1_multi_label_new(res_prob, [deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[i])] for i in range(len(test_data))]))
print("Accuracy: ",get_accuracy_multi_label_new(res_prob, [deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[i])] for i in range(len(test_data))]))

Started tokenizer loading
Used gpu 0
Instructions for updating:
Colocations handled automatically by placer.


W0303 16:39:53.724453 140273998071616 deprecation.py:323] From /home/maryia_pavlovets/.conda/envs/maryia_pavlovets_env/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0303 16:39:55.072811 140273998071616 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f93380df898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

INFO:tensorflow:Running local_init_op.


I0303 16:49:41.575302 140273998071616 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0303 16:49:41.642377 140273998071616 session_manager.py:493] Done running local_init_op.


F1 measure:  0.815426356589147
Accuracy:  0.8511627906976744


In [16]:
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
from bert_models import base_bert_model_interventions
base_bert_model_interventions = reload(base_bert_model_interventions)

from sklearn.utils import shuffle

common_folder = "/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3"
_intervention_labeler = base_bert_model_interventions.BaseBertModelInterventions(
    common_folder,
    [0 for i in range(11)], label_column="label",
    multilabel=True
)

def prepare_labelled_data(train_data):
    train_copy = train_data.copy()
    train_copy["label"] = ""
    for i in range(len(train_copy)):
        real_labels = deduplicated2labels[normalize_full(train_copy["Narrow concept"].values[i])]
        real_labels_idx = []
        for idx in range(1, 12):
            real_labels_idx.append(int(idx in real_labels))
        train_copy["label"].values[i] = real_labels_idx
    return train_copy

train_copy_df = prepare_labelled_data(train_df)
train_copy_df = shuffle(train_copy_df)

_intervention_labeler.train_model(train_copy_df, [])

Started tokenizer loading
Used gpu 0
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0303 16:58:02.063705 140273998071616 saver.py:1483] Saver not created because there are no variables in the graph to restore


Model partly loaded
Tokenizrer loaded
0
INFO:tensorflow:Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f93080d0e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0303 16:58:02.454256 140273998071616 estimator.py:201] Using config: {'_model_dir': '/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f93080d0e48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 8597


...

INFO:tensorflow:Saving checkpoints for 1611 into /hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3/model.ckpt.


I0303 17:08:35.572641 140273998071616 basic_session_run_hooks.py:594] Saving checkpoints for 1611 into /hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3/model.ckpt.


INFO:tensorflow:Loss for final step: 0.032665957.


I0303 17:08:37.428550 140273998071616 estimator.py:359] Loss for final step: 0.032665957.


Training took time  0:10:33.667194


In [17]:
res = _intervention_labeler.predict_for_df(pd.DataFrame(["faeces management", "fecal sludge management",
                                                        "disposal of feces", "faeces disposal",
                                                        "menstrual and fecal management"], columns=["text"]))

INFO:tensorflow:Writing example 0 of 5


I0303 17:10:49.847390 140273998071616 base_bert_model.py:478] Writing example 0 of 5


INFO:tensorflow:*** Example ***


I0303 17:10:49.849696 140273998071616 base_bert_model.py:454] *** Example ***


INFO:tensorflow:guid: 


...

I0303 17:10:52.626985 140273998071616 saver.py:1270] Restoring parameters from /hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3/model.ckpt-1611


INFO:tensorflow:Running local_init_op.


I0303 17:10:53.269925 140273998071616 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0303 17:10:53.336988 140273998071616 session_manager.py:493] Done running local_init_op.


In [10]:
from interventions_labeling_lib import usaid_intervention_labels
from sklearn.metrics import classification_report, confusion_matrix
for i in range(11):
    print(usaid_intervention_labels.USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[i+1])
    pred = []
    for j in range(len(test_copy)):
        if res_prob[j][i] >= 0.5:
            pred.append(1)
        else:
            pred.append(0)
    real_label = []
    for j in range(len(test_copy)):
        if (i+1) in deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[j])]:
            real_label.append(1)
        else:
            real_label.append(0)
    print(confusion_matrix(real_label,pred))
    print(classification_report(real_label,pred))

Water infrastructure
[[681  19]
 [ 27 133]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.97       700
           1       0.88      0.83      0.85       160

   micro avg       0.95      0.95      0.95       860
   macro avg       0.92      0.90      0.91       860
weighted avg       0.95      0.95      0.95       860

Sanitation/Hygiene
[[724   5]
 [ 11 120]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       729
           1       0.96      0.92      0.94       131

   micro avg       0.98      0.98      0.98       860
   macro avg       0.97      0.95      0.96       860
weighted avg       0.98      0.98      0.98       860

Fecal sludge management
[[852   0]
 [  6   2]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       852
           1       1.00      0.25      0.40         8

   micro avg       0.99      0.99      0.99      

In [13]:
for j in range(len(test_copy)):
    if 3 in deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[j])]:
        print(test_copy["Narrow concept"].values[j], res_prob[j])

Fecal Sludge Management [0.03839445 0.10422119 0.54730266 0.18459827 0.18260509 0.13557991
 0.1123758  0.06305346 0.05250514 0.06351808 0.16914055]
monitoring for fecal indicator bacteria [0.03643382 0.09735352 0.41170263 0.20595396 0.43836117 0.24029815
 0.05105585 0.08080688 0.90769804 0.03098667 0.0675917 ]
faeces care [0.01235247 0.04291916 0.16378152 0.07731697 0.06385469 0.09786215
 0.04165897 0.08966142 0.01817471 0.01769477 0.17822203]
feces/menstrual management [0.06695381 0.0890308  0.26490396 0.80750036 0.14591327 0.14034548
 0.0426245  0.10938439 0.08584037 0.04511628 0.12629557]
analyzed for presence of fecal coliform [0.02951193 0.0460684  0.2090669  0.11116165 0.50014937 0.11590654
 0.03014296 0.05505002 0.83953553 0.017831   0.04008633]
handle and dispose of faeces and menstrual blood [0.07025996 0.11457908 0.28836143 0.8378805  0.18120393 0.13282639
 0.0577912  0.12578452 0.10147268 0.05185053 0.10950282]
Fecal Sludge Management [0.03839445 0.10422119 0.54730266 0.1845

In [18]:
from interventions_labeling_lib import usaid_intervention_labels
from sklearn.metrics import classification_report, confusion_matrix
for i in range(9):
    print(usaid_intervention_labels.USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[i+1])
    pred = []
    for j in range(len(test_copy)):
        if res_prob[j][i] >= 0.5:
            pred.append(1)
        else:
            pred.append(0)
    real_label = []
    for j in range(len(test_copy)):
        if (i+1) in deduplicated2labels[normalize_full(test_copy["Narrow concept"].values[j])]:
            real_label.append(1)
        else:
            real_label.append(0)
    print(confusion_matrix(real_label,pred))
    print(classification_report(real_label,pred))

Water infrastructure
[[694  13]
 [ 17 136]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       707
           1       0.91      0.89      0.90       153

   micro avg       0.97      0.97      0.97       860
   macro avg       0.94      0.94      0.94       860
weighted avg       0.96      0.97      0.96       860

Sanitation/Hygiene
[[693   9]
 [ 12 146]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       702
           1       0.94      0.92      0.93       158

   micro avg       0.98      0.98      0.98       860
   macro avg       0.96      0.96      0.96       860
weighted avg       0.98      0.98      0.98       860

Water quality
[[772  17]
 [ 14  57]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       789
           1       0.77      0.80      0.79        71

   micro avg       0.96      0.96      0.96       860
   ma

In [4]:
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)
filter_word_list = text_normalizer.build_filter_dictionary(["../data/Filter_Geo_Names.xlsx"])
_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver(filter_word_list)
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

In [8]:
from interventions_labeling_lib.usaid_intervention_labels import USAIDInterventionLabels
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
from bert_models import base_bert_model_interventions
base_bert_model_interventions = reload(base_bert_model_interventions)
from time import time
import spacy
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)
import nltk

filter_word_list = text_normalizer.build_filter_dictionary(["../data/Filter_Geo_Names.xlsx"])
_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver(filter_word_list)
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

common_folder = "/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3"
_intervention_labeler = base_bert_model_interventions.BaseBertModelInterventions(
    common_folder,
    [0 for i in range(11)], label_column="label",
    multilabel=True
)
ner_model = spacy.load("/hdd2/data/maryia_pavlovets/usaid_entity-recognition-model-more-3012-169_iter")
folder_to_save = "../tmp/check_metrics_new"
os.makedirs(folder_to_save, exist_ok=True)
cnt_processed = 0
folder_to_process = "../tmp/check_metrics"
for filename in os.listdir(folder_to_process):
    print(filename)
    try:
        start_time = time()
        filename_to_save = os.path.join(folder_to_save, filename)
        if os.path.exists(filename_to_save):
            cnt_processed += 1
            print("Already processed ", filename_to_save)
            continue
        df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_to_process, filename))
        found_data_all = []
        idx = 0
        idx2doc = {}
        for i in range(len(df)):
            text = df["Text"].values[i].replace("\n", " ")
            text = re.sub("[sencmaruotv]{15,1000}", " ", text).strip()
            for sentence in nltk.sent_tokenize(re.sub("\s+", " ", text)):
                found_data = []
                results = set()
                for res in ner_model(sentence).ents:
                    intervention = _abbreviations_resolver.replace_abbreviations(text=text_normalizer.normalize_sentence(
                        res.text)).strip()
                    intervnetion_words_set = set()
                    intervnetion_words = []
                    for w in intervention.split(" "):
                        if w not in intervnetion_words_set:
                            intervnetion_words_set.add(w)
                            intervnetion_words.append(w)
                    intervention = " ".join(intervnetion_words).strip()
                    if intervention:
                        found_data.append((intervention, intervention, sentence, res.text))
                        idx2doc[idx] = i
                        idx += 1
                found_data_all.extend(found_data)
        df_to_predict = pd.DataFrame(found_data_all, columns=["Narrow concept", "Broad concepts", "Sentence", "Raw intervention"])
        res_prob, res_label, _ = _intervention_labeler.predict_for_df(df_to_predict)
        all_data_found = []
        all_probs = []
        for i in range(len(found_data_all)):
            idx = idx2doc[i]
            identified_labels = set()
            identified_labels_probs = []
            for j in range(len(res_label[i])):
                if res_label[i][j] == 1:
                    identified_labels.add(USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[j+1])
                    identified_labels_probs.append((USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[j+1], res_prob[i][j]))
            if identified_labels:
                all_data_found.append((found_data_all[i][0], found_data_all[i][2], found_data_all[i][3],
                                       list(identified_labels), identified_labels_probs, df["Page"].values[idx], df["Filename"].values[idx]))
        new_df = pd.DataFrame(all_data_found, columns=["intervention", "sentence", "raw_intervention", "intervention_labels",
                                                       "intervention_thresholds", "page", "filename"])
        print("Processed time: %.2f s" % (time() - start_time))
        excel_writer.ExcelWriter().save_df_in_excel(new_df, filename_to_save)
        cnt_processed += 1
    except Exception as err:
        print(err)

Started tokenizer loading
Used gpu 0
Model partly loaded
Tokenizrer loaded
0
Config is done
Sample_USAID_to_label_MR_02.11.21.xlsx
Read file ../tmp/check_metrics/Sample_USAID_to_label_MR_02.11.21.xlsx: 0.01s
Processed file ../tmp/check_metrics/Sample_USAID_to_label_MR_02.11.21.xlsx: 0.01s
Used for model gpu 1
Processed time: 26.00 s
Saving...
Saved to ../tmp/check_metrics_new/Sample_USAID_to_label_MR_02.11.21.xlsx


In [ ]:
from interventions_labeling_lib.usaid_intervention_labels import USAIDInterventionLabels
from bert_models import base_bert_model
base_bert_model = reload(base_bert_model)
from bert_models import base_bert_model_interventions
base_bert_model_interventions = reload(base_bert_model_interventions)
from time import time
import spacy
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)
import nltk

filter_word_list = text_normalizer.build_filter_dictionary(["../data/Filter_Geo_Names.xlsx"])
_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver(filter_word_list)
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

common_folder = "/hdd2/data/maryia_pavlovets/bert_usaid_interventions_multi_label_3"
_intervention_labeler = base_bert_model_interventions.BaseBertModelInterventions(
    common_folder,
    [0 for i in range(11)], label_column="label",
    multilabel=True
)
ner_model = spacy.load("/hdd2/data/maryia_pavlovets/usaid_entity-recognition-model-more-3012-169_iter")

for folder_to_process, folder_to_save in [("../tmp/parsed_pdfs_bart", "../tmp/parsed_pdfs_bart_extracted"),
                                          ("../tmp/parsed_pdfs_longformer", "../tmp/parsed_pdfs_longformer_extracted")]:
    os.makedirs(folder_to_save, exist_ok=True)
    cnt_processed = 0
    for filename in os.listdir(folder_to_process):
        print(filename)
        try:
            start_time = time()
            filename_to_save = os.path.join(folder_to_save, filename)
            if os.path.exists(filename_to_save):
                cnt_processed += 1
                print("Already processed ", filename_to_save)
                continue
            df = excel_reader.ExcelReader().read_df_from_excel(os.path.join(folder_to_process, filename))
            found_data_all = []
            idx = 0
            idx2doc = {}
            for i in range(len(df)):
                text = df["summary"].values[i].replace("\n", " ")
                text = re.sub("[sencmaruotv]{15,1000}", " ", text).strip()
                for sentence in nltk.sent_tokenize(re.sub("\s+", " ", text)):
                    found_data = []
                    results = set()
                    for res in ner_model(sentence).ents:
                        intervention = _abbreviations_resolver.replace_abbreviations(text=text_normalizer.normalize_sentence(
                            res.text)).strip()
                        intervnetion_words_set = set()
                        intervnetion_words = []
                        for w in intervention.split(" "):
                            if w not in intervnetion_words_set:
                                intervnetion_words_set.add(w)
                                intervnetion_words.append(w)
                        intervention = " ".join(intervnetion_words).strip()
                        if intervention:
                            found_data.append((intervention, intervention, sentence, res.text))
                            idx2doc[idx] = i
                            idx += 1
                    found_data_all.extend(found_data)
            df_to_predict = pd.DataFrame(found_data_all, columns=["Narrow concept", "Broad concepts", "Sentence", "Raw intervention"])
            res_prob, res_label, _ = _intervention_labeler.predict_for_df(df_to_predict)
            all_data_found = []
            all_probs = []
            for i in range(len(found_data_all)):
                idx = idx2doc[i]
                identified_labels = set()
                identified_labels_probs = []
                for j in range(len(res_label[i])):
                    if res_label[i][j] == 1:
                        identified_labels.add(USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[j+1])
                        identified_labels_probs.append((USAIDInterventionLabels.INTERVENTION_NUMBER_TO_LABEL[j+1], res_prob[i][j]))
                if identified_labels:
                    all_data_found.append((found_data_all[i][0], found_data_all[i][2], found_data_all[i][3],
                                           list(identified_labels), identified_labels_probs, df["Page"].values[idx], df["Filename"].values[idx]))
            new_df = pd.DataFrame(all_data_found, columns=["intervention", "sentence", "raw_intervention", "intervention_labels",
                                                           "intervention_thresholds", "page", "filename"])
            print("Processed time: %.2f s" % (time() - start_time))
            excel_writer.ExcelWriter().save_df_in_excel(new_df, filename_to_save)
            cnt_processed += 1
        except Exception as err:
            print(err)

Started tokenizer loading
Used gpu 0
Model partly loaded
Tokenizrer loaded
0
Config is done
PA00KNS3.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00KNS3.xlsx
PNABN631.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PNABN631.xlsx
PA00KR9N.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00KR9N.xlsx
PA00M6CZ.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00M6CZ.xlsx
PA00KPK2.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00KPK2.xlsx
PDAAP519.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PDAAP519.xlsx
PA00N69H.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00N69H.xlsx
278-C-17-00002.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/278-C-17-00002.xlsx
PA00MQ6F.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00MQ6F.xlsx
PA00JQT3.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00JQT3.xlsx
PA00N4J4.xlsx
Already processed  ../tmp/parsed_pdfs_bart_extracted/PA00N4J4.xlsx
PA00T

Processed time: 4.14 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/AID-278-C-17-00001_second doc.xlsx
PA00T6PD.xlsx
Read file ../tmp/parsed_pdfs_bart/PA00T6PD.xlsx: 0.04s
Processed file ../tmp/parsed_pdfs_bart/PA00T6PD.xlsx: 0.01s
Used for model gpu 1
Processed time: 10.32 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/PA00T6PD.xlsx
PA00MPD6.xlsx
Read file ../tmp/parsed_pdfs_bart/PA00MPD6.xlsx: 0.02s
Processed file ../tmp/parsed_pdfs_bart/PA00MPD6.xlsx: 0.00s
Used for model gpu 1
Processed time: 7.20 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/PA00MPD6.xlsx
PBAAA503.xlsx
Read file ../tmp/parsed_pdfs_bart/PBAAA503.xlsx: 0.00s
Processed file ../tmp/parsed_pdfs_bart/PBAAA503.xlsx: 0.00s
Used for model gpu 1
Processed time: 4.56 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/PBAAA503.xlsx
PNADR653.xlsx
Read file ../tmp/parsed_pdfs_bart/PNADR653.xlsx: 0.01s
Processed file ../tmp/parsed_pdfs_bart/PNADR653.xlsx: 0.00s
Used for model gpu 1
Processed time: 5.

Processed time: 4.64 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/PA00TRVW.xlsx
PBAAH743.xlsx
Read file ../tmp/parsed_pdfs_bart/PBAAH743.xlsx: 0.01s
Processed file ../tmp/parsed_pdfs_bart/PBAAH743.xlsx: 0.00s
Used for model gpu 1
Processed time: 6.88 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/PBAAH743.xlsx
PA00TT92.xlsx
Read file ../tmp/parsed_pdfs_bart/PA00TT92.xlsx: 0.01s
Processed file ../tmp/parsed_pdfs_bart/PA00TT92.xlsx: 0.00s
Used for model gpu 1
Processed time: 10.43 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/PA00TT92.xlsx
PDACW668.xlsx
Read file ../tmp/parsed_pdfs_bart/PDACW668.xlsx: 0.02s
Processed file ../tmp/parsed_pdfs_bart/PDACW668.xlsx: 0.01s
Used for model gpu 1
Processed time: 21.98 s
Saving...
Saved to ../tmp/parsed_pdfs_bart_extracted/PDACW668.xlsx
PDACR341.xlsx
Read file ../tmp/parsed_pdfs_bart/PDACR341.xlsx: 0.01s
Processed file ../tmp/parsed_pdfs_bart/PDACR341.xlsx: 0.00s
Used for model gpu 1
Processed time: 11.64 s
Saving...
Save